In [3]:
from __future__ import absolute_import, division

from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors

import Model
import numpy as np 

import senteval


PATH_SENTEVAL = '../'
PATH_TO_DATA = 'data'

maxLength = 50    
wordDim = 50    # 300 
num_neurons = [100,100]   # [600,600f]
extractLayer = ['X_input_second','X_input_mask_2']
kernel_reg = 0.001
batch_size = 512
merge_mode = 'concat'
checkpoint = "../checkpoints/weights-epoch-20-loss-0.27-val_loss-0.27.hdf5"  # Change 
embeddingPath = "../Embedding/word2vec.6B.50d.txt"  # Change 

# SentEval prepare and batcher
def prepare(params, samples):
    return

def batcher(params, batch):
    sentences = [' '.join(s) for s in batch]

    # Preprocessing
    sentences,_ = Model.preprocessing(sentences)

    # Word Embedding 
    embedding = KeyedVectors.load_word2vec_format(embeddingPath, binary=False)
    sentences =  Model.sentence2Vec(sentences,embedding,wordDim,maxLength)

    # Restore model 
    x = np.zeros((10,maxLength,wordDim))
    model = Model.buildModel(x,kernel_reg,num_neurons,merge_mode)
    model.load_weights(checkpoint)

    # Extract th output 
    output = Model.extractHiddenState(extractLayer,model,sentences)
    output = np.concatenate((output[0],output[1]),axis=1)
    
    # Convert sentence to embedding(One dimensional vector)
    return output


In [4]:
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': True, 'kfold': 5}
params_senteval['classifier'] = {'nhid': 0, 'optim': 'adam', 'batch_size': 64,
                                 'tenacity': 5, 'epoch_size': 4}
# Set up logger
se = senteval.engine.SE(params_senteval, batcher, prepare)
transfer_tasks = ['MR']
# ['MR', 'CR', 'MPQA', 'SUBJ','TREC', 'MRPC','SICKEntailment', 'SICKRelatedness']
results = se.eval(transfer_tasks)
print(results)

74


AssertionError: Torch not compiled with CUDA enabled